In [1]:
import pandas as pd
import numpy as np
from wrangle import wrangle
import re
import warnings
import nltk
from nltk.tokenize.toktok import ToktokTokenizer

import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
df = wrangle()

In [3]:
df.head()

,colors,keywords,manaValue,name,power,rarity,subtypes,supertypes,text,toughness
0,W,Protection,2.0,Auriok Champion,1,rare,"Human,Cleric",None,Protection from black and from red Whenever an...,1
1,W,None,4.0,Auriok Salvagers,2,uncommon,"Human,Soldier",None,{1}{1}: Return target artifact card with mana ...,4
2,W,Flying,4.0,Auriok Windwalker,2,rare,"Human,Wizard",None,Flying Tap : Attach target Equipment you contr...,3
3,W,None,2.0,Leonin Squire,2,common,"Cat,Soldier",None,"When this card enters the battlefield, return ...",2
4,W,None,4.0,Loxodon Anchorite,2,common,"Elephant,Cleric",None,Tap : Prevent the next 2 damage that would be ...,3


In [4]:

df.text[2]

'Flying Tap : Attach target Equipment you control to target creature you control.'

In [5]:

# fix errors

df['text'].replace("\n" , " ", inplace=True, regex=True)
df['text'].replace("\'" , "", inplace=True, regex=True)

df['text'][2]

'Flying Tap : Attach target Equipment you control to target creature you control.'

In [6]:
re.sub("{[A-Z]}" , "{1}", df.text[2])

'Flying Tap : Attach target Equipment you control to target creature you control.'

In [7]:
# remove resource symbols from all cards 

def tap(x):
    '''This function searches for the Tap symbol and replaces it with the word Tap '''
    return re.sub("{T}" , "Tap ", x)

def remove_mana(x):
    '''This function searches for mana symbols within the card and replaces it with a 1'''
    return re.sub("{[A-Z]}" , "{1}", x)


In [8]:
# we had to search the text for the tap symbol to effectivley use nlp and so it didnt hurt the remove mana funtion
df.text = df.text.apply(tap)

In [9]:
# we had to remove the mana symbols from the text of the card because it is leaked data 
df.text = df.text.apply(remove_mana)

In [10]:

# remove card names from all cards  
def remove_name(df):
    '''This function removes the cards name from the text to avoid overfitting and replaces it with this card'''
    for i in range(len(df)):
        df['text'][i] = re.sub(df['name'][i] , "this card", df['text'][i])
    return df

In [11]:
df.head()

,colors,keywords,manaValue,name,power,rarity,subtypes,supertypes,text,toughness
0,W,Protection,2.0,Auriok Champion,1,rare,"Human,Cleric",None,Protection from black and from red Whenever an...,1
1,W,None,4.0,Auriok Salvagers,2,uncommon,"Human,Soldier",None,{1}{1}: Return target artifact card with mana ...,4
2,W,Flying,4.0,Auriok Windwalker,2,rare,"Human,Wizard",None,Flying Tap : Attach target Equipment you contr...,3
3,W,None,2.0,Leonin Squire,2,common,"Cat,Soldier",None,"When this card enters the battlefield, return ...",2
4,W,None,4.0,Loxodon Anchorite,2,common,"Elephant,Cleric",None,Tap : Prevent the next 2 damage that would be ...,3


In [12]:
# removing name from text to avoid overfitting on classification models later 
df = remove_name(df)

In [13]:
df.head()

,colors,keywords,manaValue,name,power,rarity,subtypes,supertypes,text,toughness
0,W,Protection,2.0,Auriok Champion,1,rare,"Human,Cleric",None,Protection from black and from red Whenever an...,1
1,W,None,4.0,Auriok Salvagers,2,uncommon,"Human,Soldier",None,{1}{1}: Return target artifact card with mana ...,4
2,W,Flying,4.0,Auriok Windwalker,2,rare,"Human,Wizard",None,Flying Tap : Attach target Equipment you contr...,3
3,W,None,2.0,Leonin Squire,2,common,"Cat,Soldier",None,"When this card enters the battlefield, return ...",2
4,W,None,4.0,Loxodon Anchorite,2,common,"Elephant,Cleric",None,Tap : Prevent the next 2 damage that would be ...,3


In [14]:
a = df.text[2]
a

'Flying Tap : Attach target Equipment you control to target creature you control.'

In [15]:
a = df.text[2]
re.sub('\([^)]*\)' , "", a)

'Flying Tap : Attach target Equipment you control to target creature you control.'

In [16]:

# most common card descriptions 

freq = nltk.FreqDist(df.text)
common = freq.most_common()
common[:10]

[('None', 326),
 ('Flying', 91),
 ('Trample', 26),
 ('Swampwalk ', 21),
 ('Tap : Add {1}.', 19),
 ('Vigilance', 19),
 ('Reach ', 18),
 ('Forestwalk ', 17),
 ('Flying, vigilance', 16),
 ('Haste', 14)]

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=True)
vectorized_data = vectorizer.fit_transform(df.text) 
vocab = len(vectorizer.vocabulary_)

print(f"There are {vocab}s unique words in the vocabulary.")  

There are 1802s unique words in the vocabulary.
